In [81]:
import numpy as np
import pandas as pd
import ast
import json

In [101]:
#Import games, rosters and meta data

rosters = pd.read_csv('C:/Users/erden/OneDrive/Desktop/PFFFC/rosters.csv')

metadata = pd.read_csv('C:/Users/erden/OneDrive/Desktop/PFFFC/metadata.csv')

players_df = pd.read_csv('C:/Users/erden/OneDrive/Desktop/PFFFC/players.csv')

#Import events data
with open('C:/Users/erden/OneDrive/Desktop/PFFFC/events.json', 'r') as file:
    events = json.load(file)

In [82]:
def flatten_columns(df, columns_to_flatten):
    #Flattens nested lists or dictionaries in specified columns of a DataFrame.
    
    flattened_parts = []
    
    for col in columns_to_flatten:
        # Check if the column is properly formatted as a list of dictionaries
        if df[col].apply(lambda x: isinstance(x, str)).any():
            df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        
        # Normalize the column into separate columns
        flattened = pd.DataFrame(df[col].tolist())  # Handle lists of dictionaries
        flattened.columns = [f"{col}.{subcol}" for subcol in flattened.columns]  # Add prefix
        flattened_parts.append(flattened)
    
    # Combine the original DataFrame with the flattened columns
    return pd.concat([df.drop(columns_to_flatten, axis=1)] + flattened_parts, axis=1)

In [89]:
#Flatten lists into separate columns

rosters_columns = ['player', 'team']
rosters_df = flatten_columns(rosters, rosters_columns)

metadata_columns = ['awayTeam', 'awayTeamKit', 'competition', 'homeTeam', 'homeTeamKit', 'stadium', 'videos']
metadata_df = flatten_columns(metadata, metadata_columns)

#Filter out data from the final since this would be before the final is played
events_df = pd.json_normalize(events).query('gameId != 10517')


In [4]:
#Split the lists in possessionEvents into separate rows so there will be multiple observations for ball events with multiple possession events
events_df_exp = events_df.explode('possessionEvents')

In [5]:
#Create a dataframe of possession events and add game_event_id so we can match to ball events when necessary
possession_df = pd.json_normalize(events_df_exp['possessionEvents'])

In [51]:
possession_df['game_event_id'] = events_df_exp['game_event_id'].values

In [41]:
#Repeat the same step for grades
possession_df_exp = possession_df.explode('grades')

In [43]:
grades_df = pd.json_normalize(possession_df_exp['grades'])

In [53]:
grades_df['possession_event_id'] = possession_df_exp['id'].values